In [29]:
!pip install streamlit requests pydub transformers pyngrok


In [30]:

import streamlit as st
import requests
from pydub import AudioSegment
import io

# Configuration
SARVAM_AI_API_KEY = "35215873-d423-44c0-89ea-5e2550ce057f"
API_URL = "https://api.sarvam.ai/speech-to-text"

st.set_page_config(page_title=" Sarvam Speech-to-Text", layout="centered")
st.title("🎧 Sarvam AI Speech-to-Text")
st.markdown("Upload multiple audio files (`.wav`, `.mp3`) and transcribe them in English using Sarvam AI's STT API.")

uploaded_files = st.file_uploader(" Upload Audio Files", type=["wav", "mp3"], accept_multiple_files=True)

# Fix language and model to English
language = "en-IN"
model = "saarika:v2"

def transcribe_audio(audio_bytes, lang="en-IN", model_name="saarika:v2"):
    try:
        audio = AudioSegment.from_file(io.BytesIO(audio_bytes))
        buffer = io.BytesIO()
        audio.export(buffer, format="wav")
        buffer.seek(0)
    except Exception as e:
        st.error(f"❌ Audio conversion error: {e}")
        return None

    headers = {
        "api-subscription-key": SARVAM_AI_API_KEY
    }
    data = {
        "language_code": lang,
        "model": model_name,
        "with_timestamps": False
    }
    files = {"file": ("audio.wav", buffer, "audio/wav")}

    try:
        response = requests.post(API_URL, headers=headers, data=data, files=files)
        if response.status_code in [200, 201]:
            return response.json().get("transcript", " No transcript found.")
        else:
            st.error(f" API Error {response.status_code}: {response.text}")
            return None
    except Exception as e:
        st.error(f" Request failed: {e}")
        return None



2025-06-10 12:09:50.844 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-10 12:09:50.847 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-10 12:09:50.848 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-10 12:09:50.849 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-10 12:09:50.850 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-10 12:09:50.852 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-10 12:09:50.853 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-10 12:09:50.854 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [31]:
from transformers import pipeline
import numpy as np


sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")


def map_sentiment(score, label):
    if label.lower() == "positive":
        if score >= 0.95:
            return "excellent"
        elif score >= 0.85:
            return "good"
        else:
            return "decent"
    elif label.lower() == "negative":
        if score >= 0.95:
            return "horrible"
        elif score >= 0.85:
            return "very bad"
        else:
            return "bad"
    return "neutral"


def chunk_text(text, max_length=512):
    sentences = text.split(". ")
    chunks = []
    current_chunk = ""
    for sentence in sentences:
        if len(current_chunk) + len(sentence) < max_length:
            current_chunk += sentence + ". "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + ". "
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

# Main sentiment function
def extract_sentiment(text, return_chunks=False):
    if not text.strip():
        return {
            "label": "neutral",
            "original_label": "neutral",
            "polarity": 0.0
        }

    chunks = chunk_text(text)
    results = sentiment_pipeline(chunks)

     sentiment scores
    pos_scores = []
    neg_scores = []
    mapped_labels = []

    for res in results:
        label = res['label']
        score = res['score']
        mapped = map_sentiment(score, label)
        mapped_labels.append(mapped)

        if label.lower() == "positive":
            pos_scores.append(score)
        elif label.lower() == "negative":
            neg_scores.append(score)

    # Final decision
    avg_pos = np.mean(pos_scores) if pos_scores else 0
    avg_neg = np.mean(neg_scores) if neg_scores else 0

    if avg_pos > avg_neg:
        overall_label = "positive"
        polarity = avg_pos
    elif avg_neg > avg_pos:
        overall_label = "negative"
        polarity = avg_neg
    else:
        overall_label = "neutral"
        polarity = 0.0

    final_sentiment = map_sentiment(polarity, overall_label)

    result = {
        "label": final_sentiment,
        "original_label": overall_label,
        "polarity": polarity
    }

    if return_chunks:
        result["chunk_sentiments"] = [
            {
                "text": chunk,
                "raw": res,
                "mapped_label": map_sentiment(res['score'], res['label'])
            }
            for chunk, res in zip(chunks, results)
        ]

    return result


Device set to use cpu


SENTIMENT ANALYSIS OF THE CALL TRANSCRIPT


In [32]:
text = "Hello, actually I want to cancel the order. The behavior of the delivery boy was very rude and the quality of the product was also not good. So I will like to cancel the order and very pathetic and horrible service from Flipkart and I didn't expect this. No, no, no, I will cancel the order. You guys have become too worse. Yes, keep a look, keep a look."



sentiment = extract_sentiment(text, return_chunks=True)
print(sentiment)

{'label': 'horrible', 'original_label': 'negative', 'polarity': np.float64(0.9997701048851013), 'chunk_sentiments': [{'text': "Hello, actually I want to cancel the order. The behavior of the delivery boy was very rude and the quality of the product was also not good. So I will like to cancel the order and very pathetic and horrible service from Flipkart and I didn't expect this. No, no, no, I will cancel the order. You guys have become too worse. Yes, keep a look, keep a look..", 'raw': {'label': 'NEGATIVE', 'score': 0.9997701048851013}, 'mapped_label': 'horrible'}]}


TAGGING THE TRANSCRIPT ON THE BASIS OF ISSUE OR QUERY


In [33]:
!pip install sentence-transformers faiss-cpu --quiet

from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# 1. Define your fixed tags and example sentences (including new tags)
fixed_tags_examples = {
    "return": [
        "I want to return my order.",
        "How can I send the product back?",
        "The item arrived damaged, I need to return it.",
        "Can I get a refund for a return?",
        "I initiated a return request yesterday.",
        "Return shipping costs are confusing.",
        "Product return was not accepted.",
        "Returned the product but no update on refund.",
        "How long does a return take?",
        "I changed my mind and want to return."
    ],
    "payment": [
        "My payment failed at checkout.",
        "There was a problem with my credit card.",
        "How do I update my payment method?",
        "Payment confirmation not received.",
        "I was charged twice for one order.",
        "Transaction got declined unexpectedly.",
        "Can I get help with payment errors?",
        "Payment gateway is not working.",
        "Refund hasn't been processed to my account.",
        "I want to change payment to cash on delivery."
    ],
    "delivery": [
        "My order hasn't arrived yet.",
        "Where is my package?",
        "Delivery was late by 3 days.",
        "The delivery address was incorrect.",
        "Can I reschedule my delivery?",
        "Package got lost during shipping.",
        "Received the wrong item in delivery.",
        "Delivery person was not able to contact me.",
        "How to track my shipment?",
        "The parcel was damaged upon delivery."
    ],
    "quality_issue": [
        "The product quality is poor.",
        "I received a defective item.",
        "The material feels cheap and fragile.",
        "Product stopped working after a week.",
        "There are scratches on the surface.",
        "The color is different from what was advertised.",
        "The size does not match the description.",
        "It broke during first use.",
        "The stitching on the clothes is coming apart.",
        "I am not satisfied with the build quality."
    ],
    "behavior_issue": [
        "The delivery person was rude.",
        "Customer service was unhelpful.",
        "The agent raised their voice at me.",
        "I faced rude behavior during the call.",
        "Support team was not respectful.",
        "The representative ignored my questions.",
        "I had a bad experience with the sales staff.",
        "The service agent was impatient.",
        "They didn't follow up as promised.",
        "I felt disrespected by the support team."
    ],
    "important_issue": [
        "My account was hacked.",
        "I lost access to my account.",
        "Important payment failed without notification.",
        "There is a security breach in my profile.",
        "Urgent: I need immediate help with my order.",
        "Critical issue with the product delivery.",
        "The app crashed and lost all my data.",
        "I was billed incorrectly, need urgent resolution.",
        "My personal data was leaked.",
        "Please escalate this complaint urgently."
    ],
    "technical_issue": [
        "The website is not loading.",
        "App keeps crashing on startup.",
        "I cannot login to my account.",
        "Payment gateway throws an error.",
        "The search function is broken.",
        "I experience frequent disconnections.",
        "Error message appears when submitting form.",
        "Notifications are not working.",
        "The page layout is messed up on mobile.",
        "Video playback is buffering constantly."
    ],
    "account_issue": [
        "I need to update my profile details.",
        "How do I reset my password?",
        "My account was suspended without reason.",
        "I want to close my account.",
        "There is a problem with my account verification.",
        "I cannot access premium features.",
        "My account shows wrong order history.",
        "I did not receive the account confirmation email.",
        "Account login fails despite correct credentials.",
        "I want to link my account to social media."
    ]
}

# 2. Load sentence transformer model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# 3. Compute averaged embeddings for each tag
tag_names = []
tag_embeddings = []

for tag, examples in fixed_tags_examples.items():
    embeddings = embedder.encode(examples)
    avg_embedding = np.mean(embeddings, axis=0)
    tag_names.append(tag)
    tag_embeddings.append(avg_embedding)

tag_embeddings = np.vstack(tag_embeddings).astype('float32')

# 4. Build FAISS index
dimension = tag_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)  # L2 distance
index.add(tag_embeddings)

print(f"FAISS index built with {index.ntotal} tags.")

# 5. Define function to find top-k matching tags for a transcript
def tag_transcript(transcript, top_k=3):
    transcript_emb = embedder.encode([transcript]).astype('float32')
    distances, indices = index.search(transcript_emb, top_k)
    results = []
    for dist, idx in zip(distances[0], indices[0]):
        tag = tag_names[idx]
        score = 1 / (1 + dist)  # convert L2 dist to similarity-like score
        results.append((tag, score))
    return results

# --- Example usage ---
sample_text = "Hello, this is Shriyank, I am talking to Flipkart. Yeah, my order hasn't been delivered for two days. No, I had already contacted the seller, but it has still not been delivered. Please look forward to the issues. Yes, yes."
matches = tag_transcript(sample_text)

print("Top matching tags:")
for tag, score in matches:
    print(f"- {tag} (score: {score:.4f})")


FAISS index built with 8 tags.
Top matching tags:
- delivery (score: 0.6222)
- important_issue (score: 0.5432)
- return (score: 0.5369)


In [34]:
streamlit_code = """

from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# 1. Define your fixed tags and example sentences (including new tags)
fixed_tags_examples = {
    "return": [
        "I want to return my order.",
        "How can I send the product back?",
        "The item arrived damaged, I need to return it.",
        "Can I get a refund for a return?",
        "I initiated a return request yesterday.",
        "Return shipping costs are confusing.",
        "Product return was not accepted.",
        "Returned the product but no update on refund.",
        "How long does a return take?",
        "I changed my mind and want to return."
    ],
    "payment": [
        "My payment failed at checkout.",
        "There was a problem with my credit card.",
        "How do I update my payment method?",
        "Payment confirmation not received.",
        "I was charged twice for one order.",
        "Transaction got declined unexpectedly.",
        "Can I get help with payment errors?",
        "Payment gateway is not working.",
        "Refund hasn't been processed to my account.",
        "I want to change payment to cash on delivery."
    ],
    "delivery": [
        "My order hasn't arrived yet.",
        "Where is my package?",
        "Delivery was late by 3 days.",
        "The delivery address was incorrect.",
        "Can I reschedule my delivery?",
        "Package got lost during shipping.",
        "Received the wrong item in delivery.",
        "Delivery person was not able to contact me.",
        "How to track my shipment?",
        "The parcel was damaged upon delivery."
    ],
    "quality_issue": [
        "The product quality is poor.",
        "I received a defective item.",
        "The material feels cheap and fragile.",
        "Product stopped working after a week.",
        "There are scratches on the surface.",
        "The color is different from what was advertised.",
        "The size does not match the description.",
        "It broke during first use.",
        "The stitching on the clothes is coming apart.",
        "I am not satisfied with the build quality."
    ],
    "behavior_issue": [
        "The delivery person was rude.",
        "Customer service was unhelpful.",
        "The agent raised their voice at me.",
        "I faced rude behavior during the call.",
        "Support team was not respectful.",
        "The representative ignored my questions.",
        "I had a bad experience with the sales staff.",
        "The service agent was impatient.",
        "They didn't follow up as promised.",
        "I felt disrespected by the support team."
    ],
    "important_issue": [
        "My account was hacked.",
        "I lost access to my account.",
        "Important payment failed without notification.",
        "There is a security breach in my profile.",
        "Urgent: I need immediate help with my order.",
        "Critical issue with the product delivery.",
        "The app crashed and lost all my data.",
        "I was billed incorrectly, need urgent resolution.",
        "My personal data was leaked.",
        "Please escalate this complaint urgently."
    ],
    "technical_issue": [
        "The website is not loading.",
        "App keeps crashing on startup.",
        "I cannot login to my account.",
        "Payment gateway throws an error.",
        "The search function is broken.",
        "I experience frequent disconnections.",
        "Error message appears when submitting form.",
        "Notifications are not working.",
        "The page layout is messed up on mobile.",
        "Video playback is buffering constantly."
    ],
    "account_issue": [
        "I need to update my profile details.",
        "How do I reset my password?",
        "My account was suspended without reason.",
        "I want to close my account.",
        "There is a problem with my account verification.",
        "I cannot access premium features.",
        "My account shows wrong order history.",
        "I did not receive the account confirmation email.",
        "Account login fails despite correct credentials.",
        "I want to link my account to social media."
    ]
}

# 2. Load sentence transformer model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# 3. Compute averaged embeddings for each tag
tag_names = []
tag_embeddings = []

for tag, examples in fixed_tags_examples.items():
    embeddings = embedder.encode(examples)
    avg_embedding = np.mean(embeddings, axis=0)
    tag_names.append(tag)
    tag_embeddings.append(avg_embedding)

tag_embeddings = np.vstack(tag_embeddings).astype('float32')

# 4. Build FAISS index
dimension = tag_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)  # L2 distance
index.add(tag_embeddings)

print(f"FAISS index built with {index.ntotal} tags.")

# 5. Define function to find top-k matching tags for a transcript
def tag_transcript(transcript, top_k=3):
    transcript_emb = embedder.encode([transcript]).astype('float32')
    distances, indices = index.search(transcript_emb, top_k)
    results = []
    for dist, idx in zip(distances[0], indices[0]):
        tag = tag_names[idx]
        score = 1 / (1 + dist)  # convert L2 dist to similarity-like score
        results.append((tag, score))
    return results

# --- Example usage ---
sample_text = "Hello, this is Shriyank, I am talking to Flipkart. Yeah, my order hasn't been delivered for two days. No, I had already contacted the seller, but it has still not been delivered. Please look forward to the issues. Yes, yes."
matches = tag_transcript(sample_text)

print("Top matching tags:")
for tag, score in matches:
    print(f"- {tag} (score: {score:.4f})")






import streamlit as st
import requests
from pydub import AudioSegment
import io
import numpy as np
from transformers import pipeline



def tag_transcript(transcript, top_k=3):
    transcript_emb = embedder.encode([transcript]).astype('float32')
    distances, indices = index.search(transcript_emb, top_k)
    results = []
    for dist, idx in zip(distances[0], indices[0]):
        tag = tag_names[idx]
        score = 1 / (1 + dist)  # convert L2 dist to similarity-like score
        results.append((tag, score))
    return results

# --- Sarvam AI STT Configuration ---
SARVAM_AI_API_KEY = "35215873-d423-44c0-89ea-5e2550ce057f"
API_URL = "https://api.sarvam.ai/speech-to-text"
LANGUAGE_CODE = "en-IN"
MODEL_NAME = "saarika:v2"

# --- Load HuggingFace Sentiment Model ---
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

def map_sentiment(score, label):
    if label.lower() == "positive":
        if score >= 0.95:
            return "excellent"
        elif score >= 0.85:
            return "good"
        else:
            return "decent"
    elif label.lower() == "negative":
        if score >= 0.95:
            return "horrible"
        elif score >= 0.85:
            return "very bad"
        else:
            return "bad"
    return "neutral"

def chunk_text(text, max_length=512):
    sentences = text.split(". ")
    chunks = []
    current_chunk = ""
    for sentence in sentences:
        if len(current_chunk) + len(sentence) < max_length:
            current_chunk += sentence + ". "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + ". "
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

def extract_sentiment(text, return_chunks=False):
    if not text.strip():
        return {
            "label": "neutral",
            "original_label": "neutral",
            "polarity": 0.0
        }

    chunks = chunk_text(text)
    results = sentiment_pipeline(chunks)

    pos_scores = []
    neg_scores = []
    mapped_labels = []

    for res in results:
        label = res['label']
        score = res['score']
        mapped = map_sentiment(score, label)
        mapped_labels.append(mapped)

        if label.lower() == "positive":
            pos_scores.append(score)
        elif label.lower() == "negative":
            neg_scores.append(score)

    avg_pos = np.mean(pos_scores) if pos_scores else 0
    avg_neg = np.mean(neg_scores) if neg_scores else 0

    if avg_pos > avg_neg:
        overall_label = "positive"
        polarity = avg_pos
    elif avg_neg > avg_pos:
        overall_label = "negative"
        polarity = avg_neg
    else:
        overall_label = "neutral"
        polarity = 0.0

    final_sentiment = map_sentiment(polarity, overall_label)

    result = {
        "label": final_sentiment,
        "original_label": overall_label,
        "polarity": polarity
    }

    if return_chunks:
        result["chunk_sentiments"] = [
            {
                "text": chunk,
                "raw": res,
                "mapped_label": map_sentiment(res['score'], res['label'])
            }
            for chunk, res in zip(chunks, results)
        ]

    return result

# --- Transcription Handler ---
def transcribe_audio(audio_bytes, file_type="mp3"):
    try:
        audio = AudioSegment.from_file(io.BytesIO(audio_bytes), format=file_type)
        buffer = io.BytesIO()
        audio.export(buffer, format="wav")
        buffer.seek(0)
    except Exception as e:
        st.error(f" Audio conversion error: {e}")
        return None

    headers = {"api-subscription-key": SARVAM_AI_API_KEY}
    data = {
        "language_code": LANGUAGE_CODE,
        "model": MODEL_NAME,
        "with_timestamps": False
    }
    files = {"file": ("audio.wav", buffer, "audio/wav")}

    try:
        response = requests.post(API_URL, headers=headers, data=data, files=files)
        if response.status_code in [200, 201]:
            return response.json().get("transcript", " No transcript found.")
        else:
            st.error(f"❌ API Error {response.status_code}: {response.text}")
            return None
    except Exception as e:
        st.error(f"❌ Request failed: {e}")
        return None

# --- Streamlit App Main ---
def main():
    st.set_page_config(page_title=" Call Transcript Analyzer", layout="wide")
    st.title(" Audio Transcription + Call Transcript Analyzer")
    st.markdown("Upload `.wav` or `.mp3` files to transcribe and analyze the conversation.")

    uploaded_files = st.file_uploader(" Upload Audio Files", type=["wav", "mp3"], accept_multiple_files=True)

    if uploaded_files:
        combined_transcript = ""
        for audio_file in uploaded_files:
            file_type = audio_file.name.split('.')[-1].lower()
            st.audio(audio_file, format=f"audio/{file_type}")
            st.info(f"Transcribing `{audio_file.name}`...")

            audio_bytes = audio_file.read()
            transcript = transcribe_audio(audio_bytes, file_type)

            if transcript:
                st.markdown(f"**Transcript for `{audio_file.name}`:**")
                st.write(transcript)
                combined_transcript += transcript + "\\n\\n"
            else:
                st.error(f"Failed to transcribe `{audio_file.name}`")

        if combined_transcript.strip():
            st.markdown("---")
            st.markdown("### Combined Transcript")
            st.text_area("Combined Transcript", combined_transcript.strip(), height=250)

            if st.button("Analyze Combined Transcript"):
                with st.spinner("Analyzing..."):
                    result = extract_sentiment(combined_transcript.strip(), return_chunks=True)
                    tags = tag_transcript(combined_transcript.strip(), top_k=3)

                st.success(" Analysis Complete!")

                with st.expander(" Overall Sentiment"):
                    st.write(f"Label: **{result['label']}**")
                    st.write(f"Polarity: {result['polarity']:.2f}")

                with st.expander(" Detected Tags"):
                    for tag, score in tags:
                        st.write(f"- **{tag}** (Confidence: {score:.2f})")

                with st.expander(" Chunk-Level Sentiment Breakdown"):
                    for i, chunk_data in enumerate(result.get("chunk_sentiments", []), 1):
                        st.markdown(f"**Chunk {i}:** {chunk_data['mapped_label']} "
                                    f"({chunk_data['raw']['label']} - {chunk_data['raw']['score']:.2f})")
                        st.caption(chunk_data["text"])

if __name__ == "__main__":
    main()
"""


In [35]:
with open("/content/zapp.py", "w") as f:
    f.write(streamlit_code)

print("Streamlit app saved to /content/zapp.py")


Streamlit app saved to /content/zapp.py


In [36]:
from pyngrok import ngrok

# Replace with your actual token from https://dashboard.ngrok.com/get-started/your-authtoken
ngrok.set_auth_token("2yHRNdhTQXM6sJYfD9PdF60D6e4_44nc9gXSyLiQsNbFmWWUh")


AFTER RUNNING BELOW CELL YOU GET TWO LINKS TO DEPLOYED SITE OUT OF THOSE TWO SELECT  NgrokTunnel URL and then click visist site

In [37]:
# Run the Streamlit app in the background
!streamlit run /content/zapp.py &>/content/logs.txt &

# Wait briefly to allow app to start
import time; time.sleep(3)

# Expose the port using ngrok
# Kill existing processes if rer

# Run streamlit in background
!streamlit run /content/zapp.py &> /content/logs.txt &

# Create the tunnel (use "http" instead of "port")
public_url = ngrok.connect(addr="8510", proto="http")
print(f" Public URL: {public_url}")


 Public URL: NgrokTunnel: "https://3005-34-106-7-242.ngrok-free.app" -> "http://localhost:8510"


# ***!!!!!!!! *** RUN THE BELOW CEll ONLY WHEN YOU HAVE MORE THAN  3 STREAMLIT ADDRESES ACTIVE AND ERROR COMES THROUGH THE NGROK. AFTER RUNNING BELOW CELL RESTART THE SESSION AGAIN AND RUN ALL ABOVE CELLS EXCEPT THIS.

In [18]:
from pyngrok import ngrok

ngrok.kill()  #  This stops all running ngrok tunnels
